# Loading data

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [9]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


## Function for reweighting probability distribution

In [0]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    # renormalize the distribution
    return distribution / np.sum(distribution)

## Processing input data

In [12]:
# Length of sequence
maxlen = 60

step = 3  # sample new sequence every 3 char

sentences = []  # holds the extracted sequences
next_chars = []  # holds the targets

for i in range(0, len(text)-maxlen, step):
  sentences.append(text[i: i+maxlen])
  next_chars.append(text[i+maxlen])

print('Number of sequence : ', len(sentences))

Number of sequence :  200278


### Vectorization

In [0]:
chars = sorted(list(set(text)))

In [14]:
chars = sorted(list(set(text)))
print('Unique Chars: ', len(chars))

Unique Chars:  57


In [15]:
char_indices = dict((char, chars.index(char)) for char in chars)
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 'ä': 53,
 'æ': 54,
 'é': 55,
 'ë': 56}

In [0]:
# Shape of the input = (sentences, maxlen_input, length_chars)
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Building the network

In [0]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer = optimizer)

## Training the model

In [0]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
import random
import sys
for epoch in range(1, 60):
  print('epoch', epoch)
  model.fit(x, y, batch_size=128, epochs=1)
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  print('--- Generating with seed: "' + generated_text + '"')
  
  for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)
    
    for i in range(400):
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)

epoch 1
Epoch 1/1
139776/200278 [===================>..........] - ETA: 48s - loss: 2.0885

200278/200278 [==============================] - 160s 800us/step - loss: 1.9865
--- Generating with seed: "ary to a real religious life (alike for its
favourite micros"
------ temperature: 0.2
ary to a real religious life (alike for its
favourite microstion of the many the menserver, and the despentions of the master and consequently of the soul of the same the many and the same the many of the many and the same the believe the many and the same the many of the many many and the many and the many and the many the many the same the many more the ent of the same the many for the consequently and the master the many of the same more of the same the------ temperature: 0.5
tly and the master the many of the same more of the same the countions and sensents of the same the same and the sense of the langerand, who the races loge he the many in
refection of the same of the belief for the discontrans of the from the mander and strange of the same the painer and
to is says in confuction of the wi

d many in very interity of belong of the same his consempleness of the interpering and belange of ------ temperature: 1.0
he same his consempleness of the interpering and belange of be will rite at counset to consecien" of this
are that re
dights for to
decomanion
extrared, as comesqualite, the indain paiss former most long time of the lonch, lighgoured
and themselves, has yof that, to love sfmen derigiously momaniating spaaniens of the hard sugqual to the first for elitioss--but a a meansh with life the roodity? , many from
door prophmincise cain: lofe--called ebelulieanes t------ temperature: 1.2
any from
door prophmincise cain: lofe--called ebelulieanes the onr undis grues
or doropurt
paxtiunsely ranknes
in uavigyood. if to asso that
one
itself en to time. bictes some cradicisonsouts, betsome pechifection of the smosto no litrath popurror mure, in this outhur of
ods from diver. which atity, that watermamomationen nately hintolly tousuneses-minbeccly; the tur on men, of same docgus
i

173824/200278 [=========================>....] - ETA: 21s - loss: 1.6349

200278/200278 [==============================] - 160s 799us/step - loss: 1.6314
--- Generating with seed: "atic, or rather, very plebeian age, "education" and "culture"
------ temperature: 0.2
atic, or rather, very plebeian age, "education" and "cultured the depossess of the conditions of the souls of the souls and all the self-condection of the soul and in the selfisted the selfisted and the souls of the self-depontained and and the selfisted and the selfistic of the selfisted and in the condection of the selfisted the condection of the soul as and and all the selfisted with the souls of the selfisted and and the selfisted and also the domine t------ temperature: 0.5
of the selfisted and and the selfisted and also the domine the souls and reason of which the defined that is a god, and the possess of the most been of his soul the "to been extraint, and for the hought of the free imaginated of the most and their acts of the some the feelings that in the all contemption of the processed 

...

soul is the end and like there are the contemplated and attenation of the bad and ever through the superiority o------ temperature: 1.0
and attenation of the bad and ever through the superiority or the fancess for the incase stistand, even strong to form shajoction of a higher. among that the weo"
le, as a hatiess? it is natura-is with
mischeitousishceness! soorivism to preamed laistophus all upon, "everomy from our teld
all
unferishessom asises aavation of yets miprdainly thit is
its someterality,
and forigulation, in
the sacrifiest,
as they man soumbles at idetly spiritual before, modert------ temperature: 1.2
est,
as they man soumbles at idetly spiritual before, modert comparomant in ghypen erragitabition,"
and gulaable and
"him--thereklof in mychearies, thater; meens. as it is go,"-efficre time this rapeable, hard grow not be sympathy sorr godory. in them; to "els adn avousish and soulfjuty heart; the
sliilly. there ded theic, it not unwerach something
in a then anotkions, in anowt

170624/200278 [========================>.....] - ETA: 23s - loss: 1.4433

200278/200278 [==============================] - 160s 799us/step - loss: 1.4432
--- Generating with seed: "iences exist. for psychology
is once more the path to the fu"
------ temperature: 0.2
iences exist. for psychology
is once more the path to the future, the superiority and the same presence of the states and there is a sense of the subject the superiority of the same whom they conscience of the states and the souls and the same possible the sense of the sense of the same the good of the same possible the self-concerns to the subject to the sense of the seriousness of the states and the same presence of the sense of the condition of the same------ temperature: 0.5
 the same presence of the sense of the condition of the same possible to the perhaps there is a has experience and assimitual moral to the such a point of the fear of all the conduct them, for the look to a proser the suld of the recognized and also be though the great very most virtue of father them our own instance, the

d by necessity. that obsage is idea an osthural, ala peeser necessity everyle, bothtable that other realm, there is take grow of disle-arded, chrisis expect, menuare
truth, which persuctively had truth, and formerly of "germenidation haffection of the has the
abre. do notsness and commandreding of their own your teluse,
notation to a confering of itre also
something as for enough
ages------ temperature: 1.2
ion to a confering of itre also
something as for enough
ages,
who a redelity as the extente truch, in nevation do:
so womad cordifinged; nothered man: there
one to or betweat
have look and which is not his self-prateed, whole that
there spirit and clusuitlely slards" to bespers of his loust--k and grarfricuus churdic, "truth-lo; far
above long peace us wpo cause".


us at ay he who hay assing tolendised. rogachly and , rirch will even to-cdeolent, curies,"epoch 7
Epoch 1/1
  8704/200278 [>.............................] - ETA: 2:33 - loss: 1.4047

115712/200278 [================>.............] - ETA: 1:07 - loss: 1.4188